In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# if you mount Google drive correctly, the following commands should be able to be executed correctly
!ls /content/drive/
%cd "/content/drive/Shared drives/computer vision 2019"

'My Drive'  'Shared drives'
/content/drive/Shared drives/computer vision 2019


In [3]:
!ls 

 codes
 dataset
' Final Project Proposal Presentation Slide .gdoc'
 logging.gsheet
 papers
'Presentation slide.gslides'
 proposal.gdoc
'proposal presentation notices.gdoc'
 results
 scripts
 SHM
'SHM with Style Transfer Proposal Presentation.gslides'


In [4]:
! pip install PyQt5
! pip install QtWidgets

     |████████████████████████████████| 63.5MB 60kB/s 
     |████████████████████████████████| 266kB 43.9MB/s 
ERROR: Could not find a version that satisfies the requirement QtWidgets (from versions: none)
ERROR: No matching distribution found for QtWidgets


In [0]:
import sys,os
import numpy as np
import cv2
import torch
#Qt5
from PyQt5 import QtCore, QtWidgets
from PyQt5.QtGui import QImage,QIcon,QPixmap
from PyQt5.QtWidgets import QFileDialog,QMessageBox
#Qt5
from PyQt5.QtWidgets import (QWidget, QLabel, QPushButton, QApplication, QHBoxLayout, QVBoxLayout)
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import QThread, pyqtSignal, Qt

#from AIMakeup import Makeup,Face,Organ,NoFace
#from Mode_model import GAN
%run ./codes/GUI/AIMakeup

#%run ./codes/GUI/Mode_model


In [0]:
# define thread
class Thread(QThread):
    changePixmap2 = pyqtSignal(QPixmap)
    changePixmap3 = pyqtSignal(QPixmap)
    
    def __init__(self,ModelStyle,device,parent = None):
        self.DEVICE = device
        self.ModelStyle=ModelStyle
        self.G = GAN(ModelStyle,device) 
        QThread.__init__(self, parent = parent)
    def run(self):
        cap = cv2.VideoCapture(0)
        while True:
            ret, img = cap.read()
            if ret:
                if self.ModelStyle != 0:
                    p = self.G.Transfer(img)
                    img = self.G.cv2QtFormat(img)
                    p = self.G.cv2QtFormat(p)
                    self.changePixmap2.emit(img)
                    self.changePixmap3.emit(p)
                    



In [0]:
class Ui_MainWindow(object):
    def __init__(self, MainWindow):
        self.device = torch.device("cuda:0"  if torch.cuda.is_available() else 'cpu')
        
        self.window=MainWindow
        self._setupUi()
        #控件分组
        self.bg_edit=[self.bt_brightening,self.bt_whitening,self.bt_sharpen,self.bt_smooth]
        self.bg_op=[self.bt_confirm,self.bt_cancel,self.bt_reset]
        self.bg_mode = [self.bt_segment,self.bt_beautify,self.bt_ST]
        self.bg_style = [self.bt_vangogh]
        self.bg_result=[self.bt_view_compare,self.bt_save,self.bt_save_compare]
        self.sls=[self.sl_brightening,self.sl_sharpen,self.sl_whitening,self.sl_smooth]
        #用于显示图片的标签
        self.label=QtWidgets.QLabel(self.window)
        self.sa.setWidget(self.label)
        #批量设置状态
        self._set_statu(self.bg_edit,False)
        self._set_statu(self.bg_op,False)
        self._set_statu(self.bg_mode,False)
        self._set_statu(self.bg_style,False)
        self._set_statu(self.bg_result,False)
        self._set_statu(self.sls,False)
        #导入dlib模型文件
        if os.path.exists("./shape_predictor_68_face_landmarks.dat"):
            self.path_predictor=os.path.abspath("./shape_predictor_68_face_landmarks.dat")
        else:
            QMessageBox.warning(self.centralWidget,'Warning','The default path of dlib module does not exist. Please check the file path')
            self.path_predictor,_=QFileDialog.getOpenFileName(self.centralWidget,'Please the file of dlib module','./','Data Files(*.dat)')
        #实例化化妆器
        self.mu=Makeup(self.path_predictor)
        
        self.path_img=''
        self._set_connect()
    def _set_connect(self):
        '''
        设置程序逻辑
        '''
        self.bt_open.clicked.connect(self._open_img)
            
        for op in ['sharpen','whitening','smooth','brightening','cancel','confirm','reset','save','save_compare','view_compare','vangogh','segment','beautify','ST']:
            self.__getattribute__('bt_'+op).clicked.connect(self.__getattribute__('_'+op))
            
    def _open_img(self):
        '''
        打开图片
        '''
        self.path_img,_=QFileDialog.getOpenFileName(self.centralWidget,'Open Image','./','Image Files(*.png *.jpg *.bmp)')
        if self.path_img and os.path.exists(self.path_img):
            print(self.path_img)
            self.im_bgr,self.temp_bgr,self.faces=self.mu.read_and_mark(self.path_img)
            self.im_ori,self.previous_bgr=self.im_bgr.copy(),self.im_bgr.copy()
            #self._set_statu(self.bg_edit,True)
            self._set_statu(self.bg_op,True)
            self._set_statu(self.bg_mode,True)
            #self._set_statu(self.bg_style,True)
            self._set_statu(self.bg_result,True)
            self._set_statu(self.sls,True)
            self._set_img()
            
        else:
            QMessageBox.warning(self.centralWidget,'Invalid Path','Invalid Path: reset the Image path.')
            
    def _cv2qimg(self,cvImg):
        '''
        将opencv的图片转换为QImage
        '''
        height, width, channel = cvImg.shape
        bytesPerLine = 3 * width
        return QImage(cv2.cvtColor(cvImg,cv2.COLOR_BGR2RGB).data, width, height, bytesPerLine, QImage.Format_RGB888)
        
    def _set_img(self):
        '''
        显示pixmap
        '''
        self.label.setPixmap(QPixmap.fromImage(self._cv2qimg(self.temp_bgr)))

    def _set_statu(self,group,value):
        '''
        批量设置状态
        '''
        [item.setEnabled(value) for item in group]
        
    def _confirm(self):
        '''
        确认操作
        '''
        self.im_bgr[:]=self.temp_bgr[:]
        
    def _cancel(self):
        '''
        还原到上一步
        '''
        self.temp_bgr[:]=self.previous_bgr[:]
        self._set_img()
        
    def _reset(self):
        '''
        重置为原始图片
        '''
        self.temp_bgr[:]=self.im_ori[:]
        self._set_img()
        
    def _mapfaces(self,fun,value):
        '''
        对每张脸进行迭代操作
        '''
        self.previous_bgr[:]=self.temp_bgr[:]
        for face in self.faces[self.path_img]:
            fun(face,value)
        self._set_img()
    #====== mode ================#
    def _beautify(self):
        self._set_statu(self.bg_edit,True)
        self._set_statu(self.bg_style,False)
    def _ST(self):
        self._set_statu(self.bg_style,True)
        self._set_statu(self.bg_edit,False)
    #====== Segmentation ========#
    def _segment(self):
        '''
        Input: self.Img_bgr
        output:self.tmp_bgr
        '''
        return
    #====== Style Transfer ======#
    def _vangogh(self):
        G = GAN(ModelStyle=0,device=self.sevice)
        
        
    #====== Beautify ======#     
    def _sharpen(self):
        value=min(1,max(self.sl_sharpen.value()/200,0))
        print(value)
        def fun(face,value):
            face.organs['left eye'].sharpen(value,confirm=False)
            face.organs['right eye'].sharpen(value,confirm=False)
        self._mapfaces(fun,value)
        
    def _whitening(self):
        value=min(1,max(self.sl_whitening.value()/200,0))
        print(value)
        def fun(face,v):
            face.organs['left eye'].whitening(value,confirm=False)
            face.organs['right eye'].whitening(value,confirm=False)
            face.organs['left brow'].whitening(value,confirm=False)
            face.organs['right brow'].whitening(value,confirm=False)
            face.organs['nose'].whitening(value,confirm=False)
            face.organs['forehead'].whitening(value,confirm=False)
            face.organs['mouth'].whitening(value,confirm=False)
            face.whitening(value,confirm=False)
        self._mapfaces(fun,value)

    def _brightening(self):
        value=min(1,max(self.sl_brightening.value()/200,0))
        print(value)
        def fun(face,value):
            face.organs['mouth'].brightening(value,confirm=False)
        self._mapfaces(fun,value)
        
    def _smooth(self):
        value=min(1,max(self.sl_smooth.value()/100,0))
        def fun(face,value):
            face.smooth(value,confirm=False)
            face.organs['left eye'].smooth(value*2/3,confirm=False)
            face.organs['right eye'].smooth(value*2/3,confirm=False)
            face.organs['left brow'].smooth(value*2/3,confirm=False)
            face.organs['right brow'].smooth(value*2/3,confirm=False)
            face.organs['nose'].smooth(value*2/3,confirm=False)
            face.organs['forehead'].smooth(value*3/2,confirm=False)
            face.organs['mouth'].smooth(value,confirm=False)
        self._mapfaces(fun,value)
    #======================#
            
    
    def _save(self):
        output_path,_=QFileDialog.getSaveFileName(self.centralWidget,'Save to','./','Image Files(*.png *.jpg *.bmp)')
        if output_path:
            self.save(output_path,self.im_bgr)
        else:
            QMessageBox.warning(self.centralWidget,'Invalid Path','Invalid Path!! Please choose another path.')
        
    def _save_compare(self):
        output_path,_=QFileDialog.getSaveFileName(self.centralWidget,'Save comparation Img. to:','./','Image Files(*.png *.jpg *.bmp)')
        if output_path:
            self.save(output_path,np.concatenate([self.im_ori,self.im_bgr],1))
        else:
            QMessageBox.warning(self.centralWidget,'Invalid Path','Invalid Path!! Please choose another path.')
        
    def _view_compare(self):
        cv2.imshow('Compare',np.concatenate([self.im_ori,self.im_bgr],1))
        cv2.waitKey()
    def _video_thread(self):
        # define a thread
        imagePath = self.path_img
        ModelStyle = self.phtoStyle
        
        image2 = QLabel(self)   #video image
        image3 = QLabel(self)   #transfer image
        pixmap = QPixmap('Neural_Style/outputs/17.png')
        pixmap = pixmap.scaled(256, 256)
        image2.setPixmap(pixmap)
        pixmap = QPixmap('Neural_Style/outputs/out.png')
        pixmap = pixmap.scaled(256, 256)
        image3.setPixmap(pixmap)
        
        self.th = Thread(self,ModelStyle,imagePath,device=self.device)
        self.th.changePixmap2.connect(image2.setPixmap)
        self.th.changePixmap3.connect(image3.setPixmap)
        self.th.start()

        self.show()
        
    def _setupUi(self):
        self.window.setObjectName("MainWindow")
        self.window.resize(837, 838)
        self.centralWidget = QtWidgets.QWidget(self.window)
        self.centralWidget.setObjectName("centralWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralWidget)
        self.verticalLayout.setObjectName("verticalLayout")
        self.sa = QtWidgets.QScrollArea(self.centralWidget)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Expanding)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.sa.sizePolicy().hasHeightForWidth())
        self.sa.setSizePolicy(sizePolicy)
        self.sa.setWidgetResizable(True)
        self.sa.setObjectName("sa")
        self.scrollAreaWidgetContents = QtWidgets.QWidget()
        self.scrollAreaWidgetContents.setGeometry(QtCore.QRect(0, 0, 813, 532))
        self.scrollAreaWidgetContents.setObjectName("scrollAreaWidgetContents")
        self.sa.setWidget(self.scrollAreaWidgetContents)
        self.verticalLayout.addWidget(self.sa)
        self.gridLayout = QtWidgets.QGridLayout()
        self.gridLayout.setObjectName("gridLayout")
        #%% Beautify Buttons
        self.bt_whitening = QtWidgets.QPushButton(self.centralWidget)
        self.bt_whitening.setObjectName("bt_whitening")
        self.gridLayout.addWidget(self.bt_whitening, 0, 0, 1, 1)
        self.sl_whitening = QtWidgets.QSlider(self.centralWidget)
        self.sl_whitening.setOrientation(QtCore.Qt.Horizontal)
        self.sl_whitening.setObjectName("sl_whitening")
        self.gridLayout.addWidget(self.sl_whitening, 0, 1, 1, 1)
        spacerItem = QtWidgets.QSpacerItem(40, 20, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum)
        self.gridLayout.addItem(spacerItem, 0, 2, 1, 1)
        self.bt_smooth = QtWidgets.QPushButton(self.centralWidget)
        self.bt_smooth.setObjectName("bt_smooth")
        self.gridLayout.addWidget(self.bt_smooth, 1, 0, 1, 1)
        self.sl_smooth = QtWidgets.QSlider(self.centralWidget)
        self.sl_smooth.setOrientation(QtCore.Qt.Horizontal)
        self.sl_smooth.setObjectName("sl_smooth")
        self.gridLayout.addWidget(self.sl_smooth, 1, 1, 1, 1)
        spacerItem1 = QtWidgets.QSpacerItem(40, 20, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum)
        self.gridLayout.addItem(spacerItem1, 1, 2, 1, 1)
        self.bt_sharpen = QtWidgets.QPushButton(self.centralWidget)
        self.bt_sharpen.setObjectName("bt_sharpen")
        self.gridLayout.addWidget(self.bt_sharpen, 2, 0, 1, 1)
        self.sl_sharpen = QtWidgets.QSlider(self.centralWidget)
        self.sl_sharpen.setOrientation(QtCore.Qt.Horizontal)
        self.sl_sharpen.setObjectName("sl_sharpen")
        self.gridLayout.addWidget(self.sl_sharpen, 2, 1, 1, 1)
        spacerItem2 = QtWidgets.QSpacerItem(40, 20, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum)
        self.gridLayout.addItem(spacerItem2, 2, 2, 1, 1)
        self.bt_brightening = QtWidgets.QPushButton(self.centralWidget)
        self.bt_brightening.setObjectName("bt_brightening")
        self.gridLayout.addWidget(self.bt_brightening, 3, 0, 1, 1)
        self.sl_brightening = QtWidgets.QSlider(self.centralWidget)
        self.sl_brightening.setOrientation(QtCore.Qt.Horizontal)
        self.sl_brightening.setObjectName("sl_brightening")
        self.gridLayout.addWidget(self.sl_brightening, 3, 1, 1, 1)
        spacerItem3 = QtWidgets.QSpacerItem(40, 20, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum)
        self.gridLayout.addItem(spacerItem3, 3, 2, 1, 1)
        #%% Mode Button
        self.bt_segment = QtWidgets.QPushButton(self.centralWidget)
        self.bt_segment.setObjectName("bt_setment")
        self.gridLayout.addWidget(self.bt_segment, 1, 3, 1, 1)
        
        self.bt_ST = QtWidgets.QPushButton(self.centralWidget)
        self.bt_ST.setObjectName("bt_ST")
        self.gridLayout.addWidget(self.bt_ST, 2, 3, 1, 1)
        
        self.bt_beautify = QtWidgets.QPushButton(self.centralWidget)
        self.bt_beautify.setObjectName("bt_beautify")
        self.gridLayout.addWidget(self.bt_beautify, 3, 3, 1, 1)
        
        self.bt_vangogh = QtWidgets.QPushButton(self.centralWidget)
        self.bt_vangogh.setObjectName("bt_vangogh")
        self.gridLayout.addWidget(self.bt_vangogh, 4, 3, 1, 1)  #location setting(vertical,horozontal)

        #%% Panel Button
        self.bt_open = QtWidgets.QPushButton(self.centralWidget)
        self.bt_open.setObjectName("bt_open")
        self.gridLayout.addWidget(self.bt_open, 4, 0, 1, 1)
        self.bt_confirm = QtWidgets.QPushButton(self.centralWidget)
        self.bt_confirm.setObjectName("bt_confirm")
        self.gridLayout.addWidget(self.bt_confirm, 5, 0, 1, 1)
        self.bt_cancel = QtWidgets.QPushButton(self.centralWidget)
        self.bt_cancel.setObjectName("bt_cancel")
        self.gridLayout.addWidget(self.bt_cancel, 5, 1, 1, 1)
        self.bt_reset = QtWidgets.QPushButton(self.centralWidget)
        self.bt_reset.setObjectName("bt_reset")
        self.gridLayout.addWidget(self.bt_reset, 5, 2, 1, 1)
        self.bt_view_compare = QtWidgets.QPushButton(self.centralWidget)
        self.bt_view_compare.setObjectName("bt_view_compare")
        self.gridLayout.addWidget(self.bt_view_compare, 6, 0, 1, 1)
        self.bt_save = QtWidgets.QPushButton(self.centralWidget)
        self.bt_save.setObjectName("bt_save")
        self.gridLayout.addWidget(self.bt_save, 7, 1, 1, 1)
        self.bt_save_compare = QtWidgets.QPushButton(self.centralWidget)
        self.bt_save_compare.setObjectName("bt_save_compare")
        self.gridLayout.addWidget(self.bt_save_compare, 7, 2, 1, 1)
        self.verticalLayout.addLayout(self.gridLayout)
        self.window.setCentralWidget(self.centralWidget)
        
        self.retranslateUi()
        QtCore.QMetaObject.connectSlotsByName(self.window)

    def save(self,output_path,output_im):
        '''
        保存图片
        '''
        cv2.imencode('.jpg',output_im)[1].tofile(output_path)

    def retranslateUi(self): # text on the buttons
        _translate = QtCore.QCoreApplication.translate
        self.window.setWindowTitle(_translate("MainWindow", "AI "))
        self.bt_whitening.setText(_translate("MainWindow", "Whitening"))
        self.bt_smooth.setText(_translate("MainWindow", "Smooth Skin"))
        self.bt_sharpen.setText(_translate("MainWindow", "EyeFilter"))
        self.bt_brightening.setText(_translate("MainWindow", "Brighten"))
        self.bt_open.setText(_translate("MainWindow", "open Image"))
        self.bt_confirm.setText(_translate("MainWindow", "Confirm change"))
        self.bt_cancel.setText(_translate("MainWindow", "Cancel change"))
        self.bt_reset.setText(_translate("MainWindow", "Reset"))
        self.bt_view_compare.setText(_translate("MainWindow", "Before+After"))
        self.bt_save.setText(_translate("MainWindow", "SAVE"))
        self.bt_save_compare.setText(_translate("MainWindow", "Save comparison"))
        #
        self.bt_vangogh.setText(_translate("MainWindow", "ST_Vangogh"))
        self.bt_segment.setText(_translate("MainWindow", "Segment"))
        self.bt_beautify.setText(_translate("MainWindow", "BeauFace"))
        self.bt_ST.setText(_translate("MainWindow", "StyleTrans"))



In [0]:

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow(MainWindow)
    ui.window.show()
    sys.exit(app.exec_())